### Purpose

This notebook's purpose is to calculate the nudge impact of the distributions generated in the different experiments 

In [ ]:
import numpy as np
import max_nudges
import matplotlib.pyplot as plt

### Ising model

The results for the ising model using Glauber dynammics

In [ ]:
filepath = "/home/derkjan/Documents/academics_UVA/master_thesis_code/master_thesis/"

#settings for Ising using glauber dynamics
distributions_ising_glauber = "ising_distributions"
marginal_format_ising_glauber = "{}/marginal{}_neighbors{}_network_size{}_network_degree{}_temp{:.2f}.npy"
conditional_format_ising_glauber = "{}/conditional{}_neighbors{}_network_size{}_network_degree{}_temp{:.2f}.npy"
number_of_distributions_ising = 30
max_number_of_neighbors_ising = 9
number_of_nodes_ising = 100
network_degree_ising = 2
temperature_ising = 2.5


#settings for SIRS models
distributions_SIRS = "SIRS_distributions"
marginal_format_SIRS = "{}/marginal{}_neighbors{}_network_size{}_network_degree{}_mode_{}.npy"
conditional_format_SIRS = "{}/conditional{}_neighbors{}_network_size{}_network_degree{}_mode_{}.npy"
number_of_distributions_SIRS = 100
max_number_of_neighbors_SIRS = 7
number_of_nodes_SIRS = 100
network_degree_SIRS = 2
mode_SIRS = "powerlaw"


In [ ]:
distributions_folder = filepath + distributions_SIRS
filename_marginal_format = marginal_format_SIRS
filename_conditional_format = conditional_format_SIRS
number_of_distributions = number_of_distributions_SIRS
max_number_of_neighbors = max_number_of_neighbors_SIRS
number_of_nodes = number_of_nodes_SIRS
network_degree = network_degree_SIRS
temperature = temperature_ising
mode = mode_SIRS
nudge_size = 0.01


In [ ]:
distributions = []
for count in range(number_of_distributions):
    neighbors_to_distributions = {}
    for number_of_neighbors in range(1, max_number_of_neighbors):
        filename_marginal = filename_marginal_format.format(
            distributions_folder, count, number_of_neighbors, number_of_nodes, network_degree, mode
        )
        filename_conditional = filename_conditional_format.format(
            distributions_folder, count, number_of_neighbors, number_of_nodes, network_degree, mode
        )
        try:
            marginal = np.load(filename_marginal)
            conditional = np.load(filename_conditional)
            neighbors_to_distributions[number_of_neighbors] = {
                "marginal": marginal,
                "conditional": conditional
            }
        except IOError:
            print("file not found distribution number {}, number of neighbors {}".format(
                count, number_of_neighbors
            ))
    distributions.append(neighbors_to_distributions)

In [ ]:
number_of_neighbors_to_distributions = {}
for distribution in distributions:
    for number_of_neighbors, dist_dict in distribution.items():
        if number_of_neighbors in number_of_neighbors_to_distributions:
            number_of_neighbors_to_distributions[number_of_neighbors].append(dist_dict)
        else:
            number_of_neighbors_to_distributions[number_of_neighbors] = [dist_dict]
        
#print(number_of_neighbors_to_distributions[2])
        

### parameters nudge optimization

In [ ]:
nudge_size = 0.01
evolutionary_parameters = {
    "number_of_generations": 100,
    "population_size": 10,
    "number_of_children": 20,
    "generational": False,
    "mutation_size": nudge_size / 10,
    "parent_selection_mode": "rank_exponential",
    "start_mutation_size": nudge_size / 20,
    "change_mutation_size": nudge_size / (25 * 10),
    "mutations_per_update_step": 30
}


In [ ]:
nudge_type_to_impacts = {"individual":{}, "global":{}, "synergistic": {}}
number_of_neighbors_to_impacts = {}
for count, distributions in number_of_neighbors_to_distributions.items():
    print(count)
    individual_impacts = []
    synergistic_impacts = []
    global_impacts = []
    for dist_dict in distributions:
        try:
            individual_impact = max_nudges.max_nudge_impact(
                dist_dict["marginal"], dist_dict["conditional"], nudge_size, 
                "individual", None
            )
            individual_impacts.append(individual_impact)
        except:
            print("some problem occured")

#         if count >= 2:
#             synergistic_impact = max_nudges.max_nudge_impact(
#                 dist_dict["marginal"], dist_dict["conditional"], nudge_size, 
#                 "synergistic", evolutionary_parameters
#             ) 
#             synergistic_impacts.append(synergistic_impact)

        global_impact = max_nudges.max_nudge_impact(
            dist_dict["marginal"], dist_dict["conditional"], nudge_size, 
            "global", None
        ) 
        global_impacts.append(global_impact)
    
    nudge_type_to_impacts["individual"][count] = individual_impacts
#     nudge_type_to_impacts["synergistic"][count] = synergistic_impacts
    nudge_type_to_impacts["global"][count] = global_impacts
    


#max_nudges.max_nudge_impact(
#    dist_dict["marginal"], dist_dict["conditional"], nudge_size, 
#    "synergistic", evolutionary_parameters
#) 


In [ ]:
# nudge_type_to_impacts["synergistic"].pop(1)
print({k:np.mean(v) for k,v in nudge_type_to_impacts["individual"].items()})
# print({k:np.mean(v) for k,v in nudge_type_to_impacts["synergistic"].items()})
print({k:np.mean(v) for k,v in nudge_type_to_impacts["global"].items()})

In [ ]:
# nudge_types = ["individual", "synergistic", "global"]
nudge_types = ["individual", "global"]
for nudge_type in nudge_types:
    #add confidence intervals based on percentiles
    lower_bound, median, upper_bound = [], [], []
    for variable_number, impacts in nudge_type_to_impacts[nudge_type].items():
        lower_bound.append(np.percentile(impacts, 2.5))
        median.append(np.median(impacts)) 
        upper_bound.append(np.percentile(impacts, 97.5))
        
    plot_range = [i-1 for i in list(nudge_type_to_impacts[nudge_type].keys())]
    plt.plot(plot_range, median, label=nudge_type)
    plt.fill_between(plot_range, lower_bound, upper_bound, 
                     label='{}'.format("95%"), alpha=0.2)
    
plt.legend()    
plt.show()